In [13]:
import open3d as o3d
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os
import random

In [14]:
datasetPath = "/PublicData/CadDataset/a1.0.0_10"
part = "140294_f659b875"
name = "assembly.obj"
file_path = os.path.join(datasetPath,part,name)
num_sample_points = 50000          # 采样点数
normal_radius = 0.02                # 法向量估计半径
normal_max_nn = 30                  # 法向量估计最大邻居数
angle_threshold_deg = 15            # 区域生长法向量夹角阈值（度）
k_neighbors = 30                    # k-NN中的k值
min_cluster_size = 100              # 过滤小簇阈值

In [15]:
mesh = o3d.io.read_triangle_mesh(file_path)
mesh.compute_vertex_normals()

print(f"[INFO] 网格顶点数: {len(mesh.vertices)}, 面数: {len(mesh.triangles)}")
pcd = mesh.sample_points_poisson_disk(number_of_points=num_sample_points)
print(f"[INFO] 采样点数: {len(pcd.points)}")

[INFO] 网格顶点数: 111342, 面数: 122916
[INFO] 采样点数: 50000


In [16]:
# ===== 2. 估计法向量 =====
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=normal_radius, 
        max_nn=max_nn
    )
)

In [17]:
# 3. 自定义区域生长分割函数
def region_growing(pcd, angle_threshold_deg=15, k=30, min_cluster_size=100):
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)
    N = len(points)
    
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(points)
    _, indices = nbrs.kneighbors(points)
    
    labels = -np.ones(N, dtype=int)
    cluster_id = 0
    
    for i in range(N):
        if labels[i] != -1:
            continue
        queue = [i]
        labels[i] = cluster_id
        while queue:
            current = queue.pop(0)
            for neighbor_idx in indices[current][1:]:
                if labels[neighbor_idx] != -1:
                    continue
                cos_angle = np.clip(np.dot(normals[current], normals[neighbor_idx]), -1.0, 1.0)
                angle = np.arccos(cos_angle) * 180.0 / np.pi
                if angle < angle_threshold_deg:
                    labels[neighbor_idx] = cluster_id
                    queue.append(neighbor_idx)
        cluster_id += 1
    
    # 过滤小簇
    unique, counts = np.unique(labels, return_counts=True)
    for u, c in zip(unique, counts):
        if c < min_cluster_size:
            labels[labels == u] = -1
    return labels

In [18]:
# 4. 执行分割
labels = region_growing(pcd, angle_threshold_deg, k_neighbors, min_cluster_size)
num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Found {num_clusters} clusters")

Found 3 clusters


In [19]:
# 5. 给每个cluster赋随机颜色，噪声设黑色
colors = np.zeros((len(labels), 3))
for label in np.unique(labels):
    if label == -1:
        colors[labels == label] = [0, 0, 0]
    else:
        colors[labels == label] = [random.random(), random.random(), random.random()]

pcd.colors = o3d.utility.Vector3dVector(colors)

In [21]:
# 6. 保存和显示结果
o3d.io.write_point_cloud("segmented_region_growing_custom2.ply", pcd)

True